# Задание

Рассмотрите по вариантам (v3) следующие задачи оптимизации:
* Log-optimal investment strategy without the constraint $x≥0$ ([ex. 4.60, p. 209](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=223) and [10.14, p. 559](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=573));
* Equality constrained analytic centering ([p. 548](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=562));

$\to$ Equality constrained entropy maximization ([10.9, p. 558](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=572));
* Minimizing a separable function subject to an equality constraint, $f_i(x_i) = x_i^4$, $i∈\{1, ..., n\}$ ([ex. 5.4, p. 248](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=262));
* Optimal allocation with resource constraint, $f_i(x_i) = a_ie^{x_i} , a_i > 0$, $i ∈\{1, ..., n\}$ ([ex. 10.1, p. 523](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=537)).

Дана следующая задача: $f(x) = \sum_{i=1}^{n}x_i log_2 x_i \to min$.

При ограничении: $Ax=b$.

Где $x ∈ R_{++}^n$, $A ∈ R^{m*n}$

1) Исследуйте задачу на выпуклость. Запишите необходимые условия минимума и двойственную задачу. 
2) Для каждого значения размерности $n ∈ \{10, 20, ..., 100\}$ сгенерируйте $N = 100$ тестовых примеров (необходимо проверять, чтобы целевая функция на допустимом множестве была ограни-
чена снизу). В каждом случае найдите глобальный минимум, $x^∗ ∈R^n$, с помощью CVX.
3) Для каждого значения $n ∈ \{10, 20, ..., 100\}$ и для каждого тестового примера сгенерируйте 100 начальных точек. Для заданной точности $ε = 0.01$ по значению функции решите задачу с помощью прямого и двойственного метода Ньютона (стандартный метод Ньютона для решения двойственной задачи). Приведите необходимые аналитические вычисления.
4) В качестве результата работы метода:
    * Для каждого метода и значений $n ∈\{10, 20, ..., 100\}$ среднее время работы метода и среднее число итераций (усреднение проводится по всем начальным точкам и по всем тестовым примерам). Сколько арифметических операций требуется для выполнения одной итерации метода?
    * Для одного тестового примера при $n = 10$ и нескольких различных начальных точек постройте зависимость точности по значению функции от числа итераций. Сравните полученные результаты для прямого и двойственного метода.

# Настройки/Импорты

Версии важных модулей:
* cvxpy==1.4.3
* numpy==1.23.0

In [83]:
import cvxpy as cp # солвер для задач
import numpy as np # для работы с массивами

import time # для отслеживания времени выполнения
from tqdm import tqdm # для отслеживания прогресса

In [21]:
n = np.arange(10, 101, 10) # возможные значения n (число переменных в задаче ~ размерность пространства) от 10 до 100 включительно
# m = (n/2).astype(np.int32) # первая размерность матрицы A (должна быть меньше n, например, в два раза меньше n)
N = 100 # число тестовых примеров для каждого значения n
P = 100 # число начальных точек для каждого примера N
ε = 0.01 # необходимая точность

DIM = 10 # интересующая нас размерность пространства, на которой будут проходить тесты

# Вспомогательные функции

Целевая функция $f(x) = \sum_{i=1}^{n}x_i log_2 x_i \to min$, где $x ∈ R_{++}^2$, $A ∈ R^{m*n}$. <br>
Что аналогично матричному виду: $x^T log_2 x \to min$.

Её ограничение: $Ax=b$.

Производная: $∇f(x) = log_2 x + \frac{1}{ln2}$.

Гессиан: $∇^2f(x) = \frac{1}{xln2}$.

In [68]:
def func_primal(x: np.array) -> np.float32:
    """
    Функция из задачи.\n
    Parameters:
        * x: текущие значения x (в виде столбца)\n
    Returns:
        * np.float32: значение функции в точке x
    """
    # return x.T @ np.log2(x)
    # return x.T @ cp.log(x)
    # return cp.sum(x * cp.log(x))
    return cp.sum(-cp.entr(x))

In [48]:
def func_primal_grad(x: np.array) -> np.array:
    """
    Производная функции из задачи.\n
    Parameters:
        * x: текущие значения x (в виде столбца)\n
    Returns:
        * np.array: вектор-столбец градиента функции в точке x
    """
    return np.log2(x) + 1/np.log(2)

In [49]:
def func_primal_hessian(x: np.array) -> np.array:
    """
    Гессиан (матрица вторых производных) функции из задачи.\n
    Parameters:
        * x: текущие значения x (в виде столбца)\n
    Returns:
        * np.array: матрица вторых производных функции в точке x
    """
    return 1/(x*np.log(2))

In [78]:
# def constraints_primal(x: np.array, A: np.array, b: np.array) -> bool:
#     """
#     Функция для проверка решения на допустимость.\n
#     Parameters:
#         * x: текущие значения x (в виде вектора-столбца)
#         * A: матрица A
#         * b: значение ограничений\n
#     Returns:
#         * bool: True — если решение допустимо, иначе — False
#     """
#     return A @ x == b


def constraints_primal(x: np.array, A: np.array) -> bool:
    """
    Функция для проверка решения на допустимость.\n
    Parameters:
        * x: текущие значения x (в виде вектора-столбца)
        * A: матрица A\n
    Returns:
        * bool: True — если решение допустимо, иначе — False
    """
    return A @ x

# 1) Исследование задачи на выпуклость. Необходимые условия минимума. Двойственная задача.

Данная задача считается решаемой тогда и только тогда, когда $rank(A) = p < n$.

## Выпуклость.

**Определения:**
1) Функция $f(x)$ считается ***convex*** (выпуклой вниз), если для $∀x,y ∈ X ⊂ R^n, ∀γ∈$ отрузку $[0, 1]$ выполняется неравенство: $f(γx + (1-γ)y) ≤ γf(x) + (1-γ)f(y)$. Другими словами — функция выпукла, если любая хорда, соединяющая две точки функции, лежит не ниже самой функции. <br>
![Определение выпуклой вниз функции](./images/convex.png)

In [31]:
for i in range(N): # идём по числу тест-кейсов
    x, y = np.random.rand(DIM, 1), np.random.rand(DIM, 1) # случайно генерируем точки x и y в пространстве размерности DIM ((DIM, 1) — для вектора-столбца)
    γ = np.random.uniform(low=0, high=1) # случайное соотношение x и y (равномерное от 0 до 1)
    if not func_primal(γ*x+(1-γ)*y) <= γ * func_primal(x) + (1-γ) * func_primal(y): # если условие выпуклости нарушено
        raise Exception("Условие выпуклости нарушено!") # выкидываем исключение

print("Функция прошла проверку на выпуклость!")

Функция прошла проверку на выпуклость!


Под ограничением $Ax=b$ понимается пересечение плоскостей (при этом таких плоскостей меньше, чем размерность пространства, $m<n$). Каждая плоскость является выпуклым множеством. Очевидно, что пересечение выпуклых множеств будет выпуклым.

![Определение выпуклой вниз функции](./images/constraints_intersection.png)

## Условие минимума.

Необходимое условие минимума функции: если функция $f(x)$ имеет минимум в точке $х = а$, то в этой точке производная либо **равна нулю**, либо **не существует** (равна бесконечности).

## Двойственная задача.

Построим двойственную задачу по аналогии из предыдущей лабораторной работу.

Изначально целевая функция имеет вид: $f(x) = \sum_{i=1}^{n}x_i log_2 x_i = x^T log_2 x\to min$. <br>
А ограничение: $Ax=b$.

Тогда её функция Лагранжа $L(x, λ)$ имеет вид: 
* $L(x, λ) = x^T log_2 x + λ^T(Ax - b)$.

Производная функции Лагранжа $\frac{dL(x, λ)}{dx}$:
* $\frac{dL(x, λ)}{dx} = log_2 x + \frac{1}{ln2} + A^Tλ = 0$.

Выражаем значение $x^*$:
* $log_2 x^* + \frac{1}{ln2} + A^Tλ = 0$
* $log_2 x^* = -(\frac{1}{ln2} + A^Tλ)$
* $x^* = 2^{-(\frac{1}{ln2} + A^Tλ)}$

Подставляем полученное значение $x^*$ в $L(x, λ)$:
* $g(λ) = (2^{-(\frac{1}{ln2} + A^Tλ)})^T (-(\frac{1}{ln2} + A^Tλ)) + λ^T(A2^{-(\frac{1}{ln2} + A^Tλ)} - b) = -2^{-(\frac{1}{ln2}^T + λ^TA)} (\frac{1}{ln2} + A^Tλ) + λ^T(A2^{-(\frac{1}{ln2} + A^Tλ)} - b)$

Таким образом ***двойственная задача*** выглядит следующим образом:
* Целевая функция: $g(λ) = -2^{-(\frac{1}{ln2}^T + λ^TA)} (\frac{1}{ln2} + A^Tλ) + λ^T(A2^{-(\frac{1}{ln2} + A^Tλ)} - b)$.
* Ограничений нет.

Тогда производная двойственной функции $g(λ)$ по $λ$ имеет следующий вид:
* $∇g(λ) = 2^{-(\frac{1}{ln2}^T + λ^TA)} ln2 A (\frac{1}{ln2}+A^Tλ) + λ^T A 2^{-(\frac{1}{ln2} + A^Tλ)}Aln2 - b$

А гессиан:
* $∇^2g(λ) = 2^{-({\frac{1}{ln2}}^T + λ^TA)}ln2 E + A2^{-(\frac{1}{ln2} + A^Tλ)} - Aλ^T2^{-(\frac{1}{ln2} + A^Tλ)}Aln2ln2A^T$

Упростим двойственную целевую функцию и её производную с помощью замены $t_1 = \frac{1}{ln2} + A^Tλ$ и $t_2 = \frac{1}{ln2}^T + λ^TA$:
* $g(λ) = -2^{-t_2} t_1 + λ^T(A2^{-t_1} - b) \to max$
* $∇g(λ) = 2^{-t_2} ln2 A t_1 + λ^T A 2^{-t_1}Aln2 - b$

In [40]:
def func_dual(λ: np.array, A: np.array, b: np.array) -> np.float32:
    """
    Двойственная функция (задача) (построена с использованием функции Лагранжа).\n
    Parameters:
        * λ: текущие значения λ
        * A: матрица A
        * b: значение ограничений прямой задачи\n
    Returns:
        * np.float32: значение двойственной функции в точке λ
    """
    n = A.shape[1] # число переменных в прямой задаче
    ln_ = np.full(shape=(n, 1), fill_value=1/np.log(2)) # вектор-столбец 1/ln2
    t_1 = ln_ + A.T @ λ # значение первого сокращения
    t_2 = ln_.T + λ.T @ A # значение второго сокращения
    return (-2 ** (-t_2) @ t_1 + λ.T @ (A @ 2 ** (-t_1) - b))[0] # значение двойственной функции ([0] — из-за вложенности)

In [97]:
def func_dual_grad(λ: np.array, A: np.array, b: np.array) -> np.array:
    """
    Производная двойственной функции из задачи.\n
    Parameters:
        * λ: текущие значения λ
        * A: матрица A
        * b: значение ограничений прямой задачи\n
    Returns:
        * np.array: вектор-столбец градиента функции в точке λ
    """
    n = A.shape[1] # число переменных в прямой задаче
    ln = np.full(shape=(n, 1), fill_value=np.log(2)) # вектор-столбец ln2
    ln_ = np.full(shape=(n, 1), fill_value=1/np.log(2)) # вектор-столбец 1/ln2
    t_1 = ln_ + A.T @ λ # значение первого сокращения
    t_2 = ln_.T + λ.T @ A # значение второго сокращения
    return 2 ** (-t_2) @ ln * A @ t_1 + λ.T @ A @ 2 ** (-t_1) * A @ ln - b

In [98]:
func_dual_grad(λ, A, b)

array([[12.06492173],
       [21.46098572],
       [17.40212528],
       [14.3511261 ],
       [24.67837655]])

In [42]:
A = np.array([[3, 4, 1], [2, 5, 7]])
λ = np.array([[4], [8]])
λ = np.array([[5], [7]])
b = np.array([[10], [9]])

In [43]:
func_dual(λ, A, b)

array([-113.])

# 2) Генерация и решение тестовых примеров с помощью встроенных методов.

### Получаем истинные ответы от солвера.

In [79]:
data = {} # словарь под данные для теста

for dim in tqdm(n): # идём по возможному числу переменных (размерности пространства)
    m = int(dim/2) # первая размерность матрицы A = число ограничений (должна быть меньше n, например, в два раза меньше n)
    data[dim] = {i: {} for i in range(N)} # подсловарь под тест-кейсы для рассматриваемой размерности dim (получилась тройная вложенность словаря)
    for i in range(N): # идём по числу тест-кейсов
        # решаем прямую задачу с помощью солвера
        x = cp.Variable(shape=(dim, 1)) # значения переменных
        A = np.random.uniform(low=0, high=1, size=(m, dim)) # генерируем случайную симметричную положительно определённую матрицу A
        b = np.random.uniform(low=0, high=1, size=(m, 1))  # значения для ограничений
        objective = cp.Minimize(func_primal(x)) # минимизируем квадратичную функцию
        # objective = cp.Minimize((x-μ).T @ A @ (x-μ)) # как должно быть с обычной математикой (не запустить, так как CVX считает такую проблему не выпуклой)
        constraints = [constraints_primal(x, A) == b] # накладываемое ограничение — сумма квадратов переменных меньше или равна 1
        problem = cp.Problem(objective, constraints) # создаём объект решаемой задачи
        res = problem.solve(solver=cp.ECOS) # решаем поставленную проблему с помощью solver

        data[dim][i]["A"] = A # запоминаем матрицу A
        data[dim][i]["b"] = b # значения ограничений
        data[dim][i]["X opt solver"] = x.value # оптимальное значение X от встроенного солвера
        data[dim][i]["Result solver"] = res # ответ от встроенного солвера

100%|██████████| 10/10 [00:16<00:00,  1.65s/it]


In [101]:
def gradient_descent_dual(λ: np.array, A: np.array, b:np.array, η:np.float32, res_solver: np.float32, ε: np.float32) -> list:
    """
    Метод градиентного спуска для подсчёта оптимума двойственной задачи.\n
    Parameters:
        * λ: изначальное значения λ
        * A: матрица A
        * res_solver: уже полученный ответ от солвера, к которому нужно сойтись
        * ε: необходимая точность ответа\n
    Returns:
        * list: [оптимальное значение функции, оптимальное значение λ, число итераций]
    """
    iterations = 0 # счётчик итераций градиентного спуска

    res_grad_dual = func_dual(λ, A, b) # # значение начального решения для рассматриваемой стартовой точки
    while abs(res_solver - res_grad_dual) > ε: # пока не сошлись с ответом солвера
        print(iterations, abs(res_solver - res_grad_dual))
        λ = λ + η * func_dual_grad(λ, A, b) # обновляем значение λ (так как задача максимизации, то идём в сторону градиента)

        res_grad_dual = func_dual(λ, A, b) # считаем значение функции
        
        iterations += 1 # увеличиваем общее число итераций на рассматриваемой размерности dim
        η = max(η * 0.999, 0.0001) # слегка уменьшаем шаг, но не меньше 0.0001

    return [res_grad_dual, λ, iterations] # возвращаем [оптимальное значение функции, оптимальное значение λ, число итераций]

In [102]:
for dim in tqdm(n): # идём по возможному числу переменных (размерности пространства)
    m = int(dim/2) # первая размерность матрицы A = число ограничений (должна быть меньше n, например, в два раза меньше n)
    η = 0.01
    iterations = 0 # всего итераций для решения всех тест-кейсов при всех начальных точках
    time_start = time.time() # замеряем время старта рассмотрения размерности dim

    for i in range(N): # идём по числу тест-кейсов
        A = data[dim][i]["A"] # матрица А для тест-кейса
        b = data[dim][i]["b"]
        res_solver = data[dim][i]["Result solver"] # результат от солвера для тест-кейса
        
        for p in range(P): # идём по числу случайных стартовых точек
            λ = np.random.rand(m, 1) # генерируем случайное значение λ (np.array двойной вложенности) из равномерного распределения [0, 1), удоавлетворяющее ограничению λ ≥ 0
            
            iterations += gradient_descent_dual(λ, A, b, η, res_solver, ε)[2] # запоминаем число итераций, что потребовалось градиентному спуску чтобы сойтись с ответом солвера с точностью ε

    data[dim]["Average time grad dual"] = (time.time() - time_start) / (N * P) # среднее время для размерности dim за (N * p) решённых вариантов задачи
    data[dim]["Average iterations grad dual"] = iterations / (N * P) # среднее число итерации для размерности dim за (N * p) решённых вариантов задачи

  0%|          | 0/10 [00:00<?, ?it/s]

0 [inf]
1 [inf]
2 [inf]
3 [inf]
4 [inf]
5 [inf]
6 [inf]
7 [inf]
8 [inf]
9 [inf]
10 [inf]
11 [inf]
12 [inf]
13 [inf]
14 [inf]
15 [inf]
16 [inf]
17 [inf]
18 [inf]
19 [inf]
20 [inf]
21 [inf]
22 [inf]
23 [inf]
24 [inf]
25 [inf]
26 [inf]
27 [inf]
28 [inf]
29 [inf]
30 [inf]
31 [inf]
32 [inf]
33 [inf]
34 [inf]
35 [inf]
36 [inf]
37 [inf]
38 [inf]
39 [inf]
40 [inf]
41 [inf]
42 [inf]
43 [inf]
44 [inf]
45 [inf]
46 [inf]
47 [inf]
48 [inf]
49 [inf]
50 [inf]
51 [inf]
52 [inf]
53 [inf]
54 [inf]
55 [inf]
56 [inf]
57 [inf]
58 [inf]
59 [inf]
60 [inf]
61 [inf]
62 [inf]
63 [inf]
64 [inf]
65 [inf]
66 [inf]
67 [inf]
68 [inf]
69 [inf]
70 [inf]
71 [inf]
72 [inf]
73 [inf]
74 [inf]
75 [inf]
76 [inf]
77 [inf]
78 [inf]
79 [inf]
80 [inf]
81 [inf]
82 [inf]
83 [inf]
84 [inf]
85 [inf]
86 [inf]
87 [inf]
88 [inf]
89 [inf]
90 [inf]
91 [inf]
92 [inf]
93 [inf]
94 [inf]
95 [inf]
96 [inf]
97 [inf]
98 [inf]
99 [inf]
100 [inf]
101 [inf]
102 [inf]
103 [inf]
104 [inf]
105 [inf]
106 [inf]
107 [inf]
108 [inf]
109 [inf]
110 [inf]


  0%|          | 0/10 [00:08<?, ?it/s]

24553 [inf]
24554 [inf]
24555 [inf]
24556 [inf]
24557 [inf]
24558 [inf]
24559 [inf]
24560 [inf]
24561 [inf]
24562 [inf]
24563 [inf]
24564 [inf]
24565 [inf]
24566 [inf]
24567 [inf]
24568 [inf]
24569 [inf]
24570 [inf]
24571 [inf]
24572 [inf]
24573 [inf]
24574 [inf]
24575 [inf]
24576 [inf]
24577 [inf]
24578 [inf]
24579 [inf]
24580 [inf]
24581 [inf]
24582 [inf]
24583 [inf]
24584 [inf]
24585 [inf]
24586 [inf]
24587 [inf]
24588 [inf]
24589 [inf]
24590 [inf]
24591 [inf]
24592 [inf]
24593 [inf]
24594 [inf]
24595 [inf]
24596 [inf]
24597 [inf]
24598 [inf]
24599 [inf]
24600 [inf]
24601 [inf]
24602 [inf]
24603 [inf]
24604 [inf]
24605 [inf]
24606 [inf]
24607 [inf]
24608 [inf]
24609 [inf]
24610 [inf]
24611 [inf]
24612 [inf]
24613 [inf]
24614 [inf]
24615 [inf]
24616 [inf]
24617 [inf]
24618 [inf]
24619 [inf]
24620 [inf]
24621 [inf]
24622 [inf]
24623 [inf]
24624 [inf]
24625 [inf]
24626 [inf]
24627 [inf]
24628 [inf]
24629 [inf]
24630 [inf]
24631 [inf]
24632 [inf]
24633 [inf]
24634 [inf]
24635 [inf]
2463

KeyboardInterrupt: 

# 3) Реализация и тестирование метода Ньютона.